# ACTOR-CRITIC LEARNING ALGORITHM (ACLA) FOR RADIOLOGIES OF CHESTS CLASSIFICATION INTO 14 CLUSTERS

- Importing packages
- creating the actor and critic neural networks
- creating the secundary function of step and reward (environment)
- creating the RL algorithm
- Turning the images into n-vectors wich are inputs of the RL algorithm
- importing data and testing (this section isn't finished yet)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import collections
from keras.preprocessing import image
from keras import applications
from keras.applications.vgg16 import preprocess_input
from torch.distributions import Categorical
import pandas as pd

Using TensorFlow backend.


In [4]:
class Memory(object):
    def __init__(self, memory_size, array):
        self.memory_size = memory_size
        self.buffer = collections.deque(array, maxlen=self.memory_size)

    def add(self, experience):
        self.buffer.append(experience)

    def size(self):
        return len(self.buffer)

    def sample(self, batch_size):
        if batch_size > len(self.buffer):
            batch_size = len(self.buffer)
        indexes = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[i] for i in indexes]

    def clear(self):
        self.buffer.clear()

# ACTOR AND CRITIC NETWORKS

In [5]:
class AN(nn.Module):
    def __init__(self, state_size, action_size):
        super(AN, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        output = nn.functional.softmax(output)
        
        return output
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
class CN(nn.Module):
    def __init__(self, state_size, action_size):
        super(CN, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

# Environments Function (Step, Reward)

In [ ]:
def step(a, bucket):  #a is an int between 0 and 2n-1 
    
    res = list(bucket)
    
    if a < n:
        res[n+a] = bucket[a]
    else: 
        res[n+a] = 0
        
    return res

def get_nbr_zero(bucket):
    
    res = 0
    for b in bucket:
        if b == 0:
            res += 1
            
    return res

# PARAMETERS

In [17]:
GAMMA = 0.99
ALPHA = 0.5
EXPLORE = 10000
INITIAL_EPSILON = 0.1
FINAL_EPSILON = 0.0001
REPLAY_MEMORY = 2
BATCH = 1
n = 100
h = 10
    
begin_learn = False
learn_steps = 0
episode_reward = 0
scores = []

# RL algorithm

In [20]:
actor_network = AN(3*n, 2*n).to(device)
critic_network = CN(3*n, 2*n).to(device)
optimizer_critic = torch.optim.Adam(critic_network.parameters(), lr=1e-4)
optimizer_actor = torch.optim.Adam(actor_network.parameters(), lr=1e-4)

memory_replay = Memory(REPLAY_MEMORY, collections.deque())


rewards = []
    
    
def rl(input_, h, flag):
    
    n = len(input_)
    state = np.concatenate((list(input_),[0]*n, list(input_)))
    state_tensor = torch.FloatTensor(state).to(device) 
    ep_reward = 0
    hist_action = []
    
    for t in range(h):
        
        output = actor_network.forward(state_tensor) 
        output = output.argsort()
        if flag == True:
            for i in range(len(output)-1,-1,-1):
                c = int(output[i])
                if c not in hist_action:
                    action = c
                    hist_action.append(action)
                    break
            
        else: 
            for i in range(0,len(output)):
                c = int(output[i])
                if c not in hist_action:
                    action = c
                    hist_action.append(action)
                    break

        next_state = step(action, state)
        R_t = critic_network.forward(state_tensor)
        reward = (1 - get_nbr_zero(state)/n)
        ep_reward += reward
        memory_replay.add((state, next_state, action, reward))
        
        
        if memory_replay.size() > BATCH:
            
            batch = memory_replay.sample(BATCH)
            batch_state, batch_next_state, batch_action, batch_reward = zip(*batch)
            batch_state = torch.FloatTensor(batch_state).squeeze(1).to(device)
            batch_next_state = torch.FloatTensor(batch_next_state).squeeze(1).to(device)
            batch_action = torch.Tensor(batch_action).unsqueeze(1).to(device)
            batch_reward = torch.Tensor(batch_reward).unsqueeze(1).to(device)

            with torch.no_grad():
                if t < h:
                    R_t_next = critic_network.forward(batch_next_state) 
                    y = batch_reward + GAMMA*R_t_next
                else:
                    y = batch_reward 
              
            delta = y - R_t_next 
            delta_normalize = []
            for i in range(len(delta)):
                if float(delta[i]) > 0:
                    delta_normalize.append(1)
                else: 
                    delta_normalize.append(0)


            if flag == True:
                loss_actor = torch.nn.BCELoss()(torch.FloatTensor(delta_normalize),torch.FloatTensor([1]*BATCH))
            else:
                loss_actor = torch.nn.BCELoss()(torch.FloatTensor(delta_normalize), torch.FloatTensor([0]*BATCH))
            loss_actor.requires_grad = True
            optimizer_actor.zero_grad()
            loss_actor.backward()
            optimizer_actor.step()

        state = next_state
        state_tensor = torch.FloatTensor(next_state).to(device) 
        
    return (reward)

        

    

# Vectorizing the Images with a VGG16 conv network

In [ ]:
model = applications.VGG16(include_top=True)       # weights = None or weights = imagenet ? 

def vectozize(path_):
    
    img = image.load_img(path = path_, target_size=(224, 224))
    ary = image.img_to_array(img)
    ary = np.expand_dims(ary, axis=0)
    ary = preprocess_input(ary)
    features = model.predict(ary)[0]

    return features

# Exemple : Training of the Edema recognizing Agent with our Data

In [9]:
train = pd.read_csv('train.csv')
del train['Sex']
del train['Age']
del train['AP/PA']
del train['Frontal/Lateral']

In [14]:
train_sample_1 = train[train['Edema'] == 1]
train_sample_2 = train[train['Edema'] == -1]

In [ ]:
list_path_1 = train['Path'].tolist()
list_path_2 = train_sample_2['Path'].tolist()
inputs_1 = []
inputs_2 = []

for i in range (len(list_path_1)): 

    inputs_1.append(vectorize(list_path_1[i][20:]))
    
    if i%50 == 0:
        print (i)
    if i == 300:
        break
        
for i in range (len(list_path_2)): 
   
    inputs_2.append(vectorize(list_path_2[i][20:]))
    
    if i%50 == 0:
        print (i)
    if i == 300:
        break

    

In [ ]:
for input_ in inputs_1:
    rl(input_h,True)
for input_ in inputs_2:
    rl(input_h,False)

In [ ]:
inp = np.concatenate((list(inputs_1[0]),[0]*n, list(inputs_1[0])))
critic_network(torch.FloatTensor(inp))

inp_ = np.concatenate((list(inputs_2[0]),[0]*n, list(inputs_2[0])))
critic_network(torch.FloatTensor(inp_))

# KMEANS algorithm : Irelevant results

In [153]:
from sklearn.cluster import KMeans

In [135]:
vector = np.concatenate((inputs_1,inputs_2))

In [156]:
kmeans = KMeans(n_clusters=13, random_state=0).fit(inputs_3)

In [157]:
labels = kmeans.labels_


In [161]:
labels

array([6, 5, 6, ..., 4, 4, 4], dtype=int32)

In [72]:
inp = np.concatenate((list(inputs_1[0]),[0]*n, list(inputs_1[0])))
critic_network(torch.FloatTensor(inp))

tensor([0.0526], grad_fn=<AddBackward0>)